In [21]:
# Import libraries

# import Pkg
# Pkg.add("QuantileRegressions")
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using Optim
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using QuantileRegressions
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction

# Access the matplotlib module
matplotlib = pyimport("matplotlib")
mpl_inset_locator = pyimport("mpl_toolkits.axes_grid1.inset_locator")
seaborn = pyimport("seaborn")
pyimport("scienceplots")
scipy = pyimport("scipy")
np = pyimport("numpy")
skl_model_selection = pyimport("sklearn.model_selection")
seaborn.color_palette("colorblind")

PyCall.pygui(:tk)



# Set a seed for reproducibility
using Random
Random.seed!(11);

In [22]:
project(pca, Y, J=size(Y,1)) = (projection(pca)[:,1:J])' * centralize(Y, pca.mean)
reconstruct(pca, W, J=size(W,1)) = decentralize(projection(pca)[:,1:J] * W, mean(pca))

reconstruct (generic function with 7 methods)

In [24]:
y = CSV.read("../Data/Training_Data/Amery_VAF_time_series_2300_ssp_5_8.5.csv", DataFrame)
y = select(y, Not(1))
y = Matrix(y)
y = collect(y');

In [25]:
rank_data = rank(y)
println("Rank of the data matrix: ", rank_data)

Rank of the data matrix: 119


In [26]:
p = fit(PCA, y, pratio=1);

In [27]:
K = 5 # Change this to the number of PCs you want
pred_pre = project(p, y, K)
rec_pre = reconstruct(p, pred_pre, K);

In [28]:
# using Plots
# i = sample(1:size(y,2))
# Plots.plot(y[:,i], lw=2, title="i = $i")
# Plots.plot!(rec_pre[:,i], lw=2, line=:dash, color="red")

In [8]:
# using Plots
# # i = sample(1:size(y,2))
# plt = Plots.plot()
# for i = 1:size(y,2)
#     Plots.plot!(y[:,i], lw=2, color=:lightskyblue, legend = false)
#     Plots.plot!(rec_pre[:,i], lw=2, line=:dash, color="red", legend = false)
# end
# # plt

In [29]:
x = CSV.read("../Data/Training_Data/Amery_GP_Emulator_RELX_Ensemble_Filtered.csv", DataFrame);

In [30]:
vmThresh_min, vmThresh_max = minimum(x.vmThresh), maximum(x.vmThresh)
scaler_vmThresh = fit(UnitRangeTransform, x.vmThresh)
d_01 = StatsBase.transform(scaler_vmThresh, x.vmThresh)

fricExp_min, fricExp_max = minimum(x.fricExp), maximum(x.fricExp)
scaler_fricExp = fit(UnitRangeTransform, x.fricExp)
d_02 = StatsBase.transform(scaler_fricExp, x.fricExp)

mu_scale_min, mu_scale_max = minimum(x.mu_scale), maximum(x.mu_scale)
scaler_mu_scale = fit(UnitRangeTransform, x.mu_scale)
d_03 = StatsBase.transform(scaler_mu_scale, x.mu_scale)

stiff_scale_min, stiff_scale_max = minimum(x.stiff_scale), maximum(x.stiff_scale)
scaler_stiff_scale = fit(UnitRangeTransform, x.stiff_scale)
d_04 = StatsBase.transform(scaler_stiff_scale, x.stiff_scale)

gamma0_min, gamma0_max = minimum(x.gamma0), maximum(x.gamma0)
scaler_gamma0 = fit(UnitRangeTransform, x.gamma0)
d_05 = StatsBase.transform(scaler_gamma0, x.gamma0)

melt_flux_min, melt_flux_max = minimum(x.melt_flux), maximum(x.melt_flux)
scaler_melt_flux = fit(UnitRangeTransform, x.melt_flux)
d_06 = StatsBase.transform(scaler_melt_flux, x.melt_flux)

param_mins = [vmThresh_min,fricExp_min,mu_scale_min,stiff_scale_min,gamma0_min,melt_flux_min]
param_maxs = [vmThresh_max,fricExp_max,mu_scale_max,stiff_scale_max,gamma0_max,melt_flux_max]

design_scaled = DataFrame(vmThresh_scaled=d_01, fricExp_scaled=d_02, mu_scale_scaled=d_03,
                          stiff_scale_scaled=d_04, gamma0_scaled=d_05, melt_flux_scaled=d_06)

X = Matrix(design_scaled)' # matrix 6 x N

6×119 adjoint(::Matrix{Float64}) with eltype Float64:
 0.407891   0.584329  0.879853  0.35452   …  0.89069    0.435987   0.727361
 0.0356264  0.509296  0.437061  0.789839     0.0620034  0.412618   0.840317
 0.354441   0.449709  0.200242  0.311622     0.394692   0.0504651  0.712968
 0.485306   0.770992  0.664396  0.733916     0.555406   0.342915   0.709687
 0.607262   0.45634   0.784528  0.921009     0.959423   0.688045   0.451809
 0.91086    0.239113  0.835963  0.482451  …  0.11399    0.180736   0.683243

In [31]:
function scale_pcs(pc_matrix, num_pcs)
    pc_scaled = Vector{Vector{Float64}}(undef, num_pcs)
    scalers = Vector{UnitRangeTransform}(undef, num_pcs)
    pc_min = Vector{Float64}(undef, num_pcs)
    pc_max = Vector{Float64}(undef, num_pcs)
    
    for i in 1:num_pcs
        pc_values = pc_matrix[i, :]
        pc_min[i], pc_max[i] = minimum(pc_values), maximum(pc_values)
        scaler = fit(UnitRangeTransform, pc_values)
        scalers[i] = scaler
        pc_scaled[i] = StatsBase.transform(scaler, pc_values)
    end
    
    return pc_scaled, scalers, pc_min, pc_max
end

# Create a 2D array where each row represents a different PC
# You can change the number of rows (i.e., the number of PCs) dynamically
PC_scaled, scalers, PC_min, PC_max = scale_pcs(pred_pre, K);

In [32]:
function fit_gp(X_train, y_train)
    mZero = MeanLin(zeros(size(X_train, 1)))  # Mean function 
    ## GaussianProcesses.MeanZero() # Zero mean function
    kern = Mat52Ard(zeros(size(X_train, 1)), 0.001)  # Kernel function 
    ##+ GaussianProcesses.Noise(0.0) # GaussianProcesses.SE(0.0,0.001)
    logObsNoise = -4.605170185985  # Log observation noise

    gp = GaussianProcesses.GP(X_train, y_train, mZero, kern, logObsNoise)
    
#     optimize!(gp, domean = true, kern = true, noise = false, iterations = 100)
    iters = 100
    max_retries = 5
    
    for attempt in 0:5
            
        res = optimize!(gp; domean=true, kern=true, noise=false, iterations=iters)

        if Optim.converged(res)
            success = true
            break
        else
            success = false
            @warn "optimize! did not converge on attempt $(attempt+1) with $iters iterations."
            if attempt < max_retries
                iters *= 2
                @info "Retrying with $iters iterations..."
                gp = GaussianProcesses.GP(X_train, y_train, mZero, kern, logObsNoise)
            end
        end
    end
    
    @info "Successfully trained emulator in $iters iterations."
    
    return gp
end

function predict_gp_eml(gp, X_test)
    μ, var = GaussianProcesses.predict(gp, X_test)
    return μ, sqrt.(var)
end

gps = [fit_gp(X, pc_scaled) for pc_scaled in PC_scaled];

┌ Warning: optimize! did not converge on attempt 1 with 100 iterations.
└ @ Main In[32]:23
[ Info: Retrying with 200 iterations...
[ Info: Successfully trained emulator in 200 iterations.
[ Info: Successfully trained emulator in 100 iterations.
[ Info: Successfully trained emulator in 100 iterations.
┌ Warning: optimize! did not converge on attempt 1 with 100 iterations.
└ @ Main In[32]:23
[ Info: Retrying with 200 iterations...
[ Info: Successfully trained emulator in 200 iterations.
┌ Warning: optimize! did not converge on attempt 1 with 100 iterations.
└ @ Main In[32]:23
[ Info: Retrying with 200 iterations...
[ Info: Successfully trained emulator in 200 iterations.


In [33]:
num_pcs = K

#  Save the relevant variables into disk:
#@save "VAF_PCA_GP_Emulators.jld2" gps scalers PC_min PC_max p num_pcs

5

In [34]:
# RELX calibrated posterior
θ_posterior_expert = np.load("../Data/Training_Data/posterior_samples_All_Combined.npy")
println(size(θ_posterior_expert))
println(typeof(θ_posterior_expert))

(100000, 6)
{Float64}


In [35]:
# Expert priors
post_full_2015 = DataFrame(vmThresh_post=θ_posterior_expert[:,1], fricExp_post=θ_posterior_expert[:,2], 
    mu_scale_post=θ_posterior_expert[:,3], stiff_scale_post=θ_posterior_expert[:,4], 
    gamma0_post=θ_posterior_expert[:,5], melt_flux_post=θ_posterior_expert[:,6]);

Random.seed!(11)

sample_size = 5000

# Get the total number of rows in the DataFrame
total_rows = nrow(post_full_2015)
# Generate random indices to select rows
random_indices = randperm(total_rows)[1:sample_size]
# Select the subset of rows using the random indices
post_2015 = post_full_2015[random_indices, :]



vmThresh_prior = scipy.stats.truncnorm.rvs(a=-3, b=3, loc=130000, scale=50000/3, size=sample_size)
fricExp_prior = scipy.stats.trapezoid.rvs(c=0.05/0.233, d=0.18/0.233, loc=0.1, scale=0.233, size=sample_size)
mu_scale_prior = scipy.stats.truncnorm.rvs(a=-2, b=2, loc=1.0, scale=0.1, size=sample_size)
stiff_scale_prior = scipy.stats.truncnorm.rvs(a=-2, b=2, loc=1.0, scale=0.1, size=sample_size)
trunc_norm_gamma0 = scipy.stats.truncnorm(a=np.log(9620)-10, b=np.log(471000)-10, loc=10, scale=1)
gamma0_prior = np.exp(trunc_norm_gamma0.rvs(size=sample_size))
melt_flux_prior = scipy.stats.truncnorm.rvs(a=-2, b=2, loc=35, scale=11.5, size=sample_size)

prior = DataFrame(vmThresh_prior=vmThresh_prior, fricExp_prior=fricExp_prior, mu_scale_prior=mu_scale_prior, 
    stiff_scale_prior=stiff_scale_prior, gamma0_prior=gamma0_prior, melt_flux_prior=melt_flux_prior);

In [36]:
num_pcs = size(PC_scaled,1)
mu_post = [Vector{Float64}() for _ in 1:num_pcs]
sig_post = [Vector{Float64}() for _ in 1:num_pcs]

for x in eachrow(post_2015)
    p1_t = (x.vmThresh_post-vmThresh_min)/(vmThresh_max-vmThresh_min)
    p2_t = (x.fricExp_post-fricExp_min)/(fricExp_max-fricExp_min)
    p3_t = (x.mu_scale_post-mu_scale_min)/(mu_scale_max-mu_scale_min)
    p4_t = (x.stiff_scale_post-stiff_scale_min)/(stiff_scale_max-stiff_scale_min)
    p5_t = (x.gamma0_post-gamma0_min)/(gamma0_max-gamma0_min)
    p6_t = (x.melt_flux_post-melt_flux_min)/(melt_flux_max-melt_flux_min)
    
    α = [p1_t;; p2_t;; p3_t;; p4_t;; p5_t;; p6_t]'
    
    for i in 1:num_pcs
        μ, σ = predict_gp_eml(gps[i], α)
        mu =  only(μ)*(PC_max[i] - PC_min[i]) + PC_min[i]
        sig = only(σ)*(PC_max[i] - PC_min[i])
        append!(mu_post[i], mu)
        append!(sig_post[i], sig)
    end
end


In [37]:
PC_mean_pred_post = hcat(mu_post...)
PC_sig_pred_post = hcat(sig_post...)

sample_post = Matrix{Float64}(undef, 0, size(y, 1))

for (mu_sample, sigma_sample) in zip(eachrow(PC_mean_pred_post), eachrow(PC_sig_pred_post))
    # Appendix B.2 steps combined to generate multivariate normal back tranformed sample
    sample = reconstruct(p, rand.(Normal.(mu_sample,sigma_sample)), num_pcs)
    sample_post = [sample_post; sample']
end
sample_post = sample_post'
    
#Convert to Sea level contribution
present2015_post_mm_ssp5 = sample_post./(-362);

In [38]:
###Code below for projecting forward with sequences of future calibrated posteriors

In [39]:
# Function that projects sea level contribution for the sequence of calibrated posteriors 2030-2300 for a given realization(trajectory)
function sequential_prediction_func(
        posterior_dict::Dict{Int64, Matrix{Float64}},
        realization::String,
        num_pcs,
        min_PC,
        max_PC,
        X_mins,
        X_maxs,
        fit,
        vfa,
        emulators

    )
        
        sample_size = 5000
        projection_dict = Dict{Int64, Matrix{Float64}}()
        #Project forward for each posterior in dictionary
        
        for key in keys(posterior_dict)
            #println(key)
            post_full = DataFrame(vmThresh_post=posterior_dict[key][:,1], fricExp_post=posterior_dict[key][:,2], 
            mu_scale_post=posterior_dict[key][:,3], stiff_scale_post=posterior_dict[key][:,4], 
            gamma0_post=posterior_dict[key][:,5], melt_flux_post=posterior_dict[key][:,6])

            Random.seed!(11)
            # Get the total number of rows in the DataFrame
            total_rows = nrow(post_full)
            # Generate random indices to select rows
            random_indices = randperm(total_rows)[1:sample_size]
            # Select the subset of rows using the random indices
            post = post_full[random_indices, :]

            #Initialize empty arrays for saving
            mu_post = [Vector{Float64}() for _ in 1:num_pcs]
            sig_post = [Vector{Float64}() for _ in 1:num_pcs]
            
            #Predict with each PC emulator for samples of the posterior
            for x in eachrow(post)
                #scale parameters
                p1_t = (x.vmThresh_post- X_mins[1])/(X_maxs[1]-X_mins[1])
                p2_t = (x.fricExp_post- X_mins[2])/(X_maxs[2]-X_mins[2])
                p3_t = (x.mu_scale_post- X_mins[3])/(X_maxs[3]-X_mins[3])
                p4_t = (x.stiff_scale_post - X_mins[4])/(X_maxs[4]-X_mins[4])
                p5_t = (x.gamma0_post- X_mins[5])/(X_maxs[5]-X_mins[5])
                p6_t = (x.melt_flux_post- X_mins[6])/(X_maxs[6]-X_mins[6])
                
                α = [p1_t;; p2_t;; p3_t;; p4_t;; p5_t;; p6_t]'
                
                #Predict on each PC
                for i in 1:num_pcs
                    μ, σ = predict_gp_eml(emulators[i], α)
                    mu =  only(μ)*(max_PC[i] - min_PC[i]) + min_PC[i]
                    sig = only(σ)*(max_PC[i] - min_PC[i])
                    append!(mu_post[i], mu)
                    append!(sig_post[i], sig)
                end
            end

            #Save the Principal components predictions 
            PC_mean_pred_post = hcat(mu_post...)
            PC_sig_pred_post = hcat(sig_post...)

            # mkpath("../../SLR_Projection_Data/pred_mean_PC/R_$(realization)")
            # mkpath("../../SLR_Projection_Data/pred_std_PC/R_$(realization)")
            # @save "../../SLR_Projection_Data/pred_mean_PC/R_$(realization)/$(realization)-year$(key)mean_PCs.jld2" PC_mean_pred_post
            # @save "../../SLR_Projection_Data/pred_std_PC/R_$(realization)/$(realization)-year$(key)std_PCs.jld2" PC_sig_pred_post

            sample_post = Matrix{Float64}(undef, 0, size(vfa, 1))

            #Reconstruct the PC predictions to get VAF outputs
            for (mu_sample, sigma_sample) in zip(eachrow(PC_mean_pred_post), eachrow(PC_sig_pred_post))
                # Appendix B.2 steps combined to generate multivariate normal back tranformed sample
                sample = reconstruct(fit, rand.(Normal.(mu_sample,sigma_sample)), num_pcs)
                sample_post = [sample_post; sample']
            end
            sample_post = sample_post'
            sample_post_mm_ssp5 = sample_post./(-362)
            projection_dict[key] = sample_post_mm_ssp5
            mkpath("../Data/ProjectionData/R_$(realization)")
            @save "../Data/ProjectionData/R_$(realization)/$(realization)-year$(key)pred_VAF.jld2" sample_post_mm_ssp5
         

        end
    
        return nothing
    end

sequential_prediction_func (generic function with 1 method)

In [ ]:
## Use this cell to project foward for a set of trajectories of posteriors

#Realization numbers
Realizations = [i for i in 1:100];
#Misconverged realization numbers
m = [4, 15, 20, 44, 47, 67, 81, 82, 89, 100]
filter!(x -> !(x in m), Realizations)
r_filtered = [string(r) for r in Realizations];

yrs_all = collect(range(2030, step=15, length=19));

for r in realizations
    θ_posterior_fut = JLD2.load("../Data/Posterior_Data/Ninety_Posterior_Sets/R_$(r)_Posterior_Dict.jld2", "post_data");
    sequential_prediction_func(
        θ_posterior_fut,
        r,
        5,
        PC_min,
        PC_max,
        param_mins,
        param_maxs,
        p,
        y,
        gps)
    println("Done with realization $(r)...")
end